Load file


In [6]:
import datetime
import pandas as pd
import pyodbc


# Config db
server = ''
database = ''
USR = ''
PWD = ''
path_db = f'DRIVER={{SQL Server}};SERVER={server};DATABASE={database};UID={USR};PWD={PWD};'



def execute_job():
    data = datetime.datetime.now().date().strftime("%Y%m%d")
    path = rf"Z:\\Rollout\\R_{data}.csv"
    
    try:
        read_f = pd.read_csv(path)
        valid_f = True
        print('Arquivo lido com sucesso')
    except FileNotFoundError:
        valid_f = False
        print(f'Arquivo não encontrado: {path}')
    
    if valid_f:
        connection = pyodbc.connect(path_db)
        cursor = connection.cursor()

        print('Processando arquivo')
        cursor.fast_executemany = True

        desired_order = ['idcliente', 'idcaso', 'dsagencia', 'dsranking', 'DescontoAntigo', 'DescontoNovo', 'DESCONTO']
        
        # Verifique as colunas no DataFrame
        print('Colunas no DataFrame:', read_f.columns.tolist())
        read_f = read_f[desired_order]

        # Verifique valores nulos
        print('Valores nulos nas colunas:', read_f.isnull().sum())

        batch_size = 100000
    
        cursor.execute("TRUNCATE TABLE [recovery].[rollout_20240717]")
        connection.commit()
        print('Tabela truncada com sucesso')

        for start in range(0, len(read_f), batch_size):
            end = start + batch_size
            batch = read_f.iloc[start:end]
            
            data_to_insert = [
                (row['idcliente'], row['idcaso'], row['dsagencia'], row['dsranking'], 
                    row['DescontoAntigo'], row['DescontoNovo'], row['DESCONTO'])
                for index, row in batch.iterrows()
            ]

            cursor.executemany(
                """
                INSERT INTO [recovery].[rollout_20240717] 
                (IdCliente, IdCaso, DsAgencia, dsranking, DescontoAntigo, DescontoNovo, DESCONTO, DataImportacao) 
                VALUES (?, ?, ?, ?, ?, ?, ?, GETDATE())
                """,
                data_to_insert
            )

            connection.commit()
            print(f'Lote {start // batch_size + 1} inserido com sucesso')

        print('Rodou replace com sucesso')
       
    
execute_job();

Arquivo lido com sucesso
Processando arquivo
Colunas no DataFrame: ['idcliente', 'idcaso', 'dsagencia', 'dsranking', 'DescontoAntigo', 'DescontoNovo', 'DESCONTO']
Valores nulos nas colunas: idcliente         0
idcaso            0
dsagencia         0
dsranking         0
DescontoAntigo    0
DescontoNovo      0
DESCONTO          0
dtype: int64
Tabela truncada com sucesso
Lote 1 inserido com sucesso
Lote 2 inserido com sucesso
Lote 3 inserido com sucesso
Rodou replace com sucesso


In [2]:
data = datetime.datetime.now().date().strftime("%Y%m%d")
path = rf"Z:\\Rollout\\R_{data}.csv"

pd.read_csv(path)

,idcliente,idcaso,dsagencia,dsranking,DescontoAntigo,DescontoNovo,DESCONTO
0,567712,47245635,Aval Administração de Cobrança e Cadastro LTDA,Coincidentes REC,46.49,67.77,Aumentou
1,583152,17032192,Aval Administração de Cobrança e Cadastro LTDA,Coincidentes REC,72.67,86.02,Aumentou
2,611956,68688525,Aval Administração de Cobrança e Cadastro LTDA,Não Bancos - Atraso > 3 Anos,57.04,60.26,Aumentou
3,662833,19821422,Aval Administração de Cobrança e Cadastro LTDA,Coincidentes REC,96.25,97.87,Aumentou
4,685640,26719735,Aval Administração de Cobrança e Cadastro LTDA,Coincidentes REC,93.50,97.87,Aumentou
...,...,...,...,...,...,...,...
270867,51527261,72132677,Aval Administração de Cobrança e Cadastro LTDA,Coincidentes REC,22.05,55.39,Aumentou
270868,51922365,71948290,Aval Administração de Cobrança e Cadastro LTDA,Coincidentes REC,65.59,75.36,Aumentou
270869,51939803,71584762,Aval Administração de Cobrança e Cadastro LTDA,Não Bancos - Atraso > 3 Anos,44.56,45.53,Aumentou
270870,52345932,72429207,Aval Administração de Cobrança e Cadastro LTDA,Coincidentes REC,76.67,84.82,Aumentou
